# Advanced Scenario Examples

This notebook demonstrates how to use the `Scenario` object within the pyetm package to retrieve and 
inspect data from an ETM scenario. These examples are slightly more advanced, including analytics and
data operations on the scenario object and its sub-models.

Make sure you have a valid `ETM_API_TOKEN` set in your environment.

## Structure

This notebook is organized into two main sections:
1. **Setup & Initialization** - Run these cells first to set up your environment and load a scenario
2. **Exploration Examples** - After setup is complete, these cells can be run in any order to explore different aspects of scenario data

## Setup & Initialization

**Run these cells first!** The following cells set up your environment and load a scenario. Complete this section before exploring the examples below.

##### Environment Setup

In [ ]:
from example_helpers import setup_notebook
from pyetm.models import Scenario

setup_notebook()

##### Load a scenario

This cell connects to a specific scenario using its session ID and loads all its data.

The scenario object will contain inputs, outputs, custom curves, and other configuration data.

In [ ]:
# Connect to your scenario by supplying the session ID
scenario = Scenario.load(2690288)

print(f"  Scenario {scenario.id} loaded successfully")
print(f"  Total inputs: {len(scenario.inputs)}")
print(f"  User-modified inputs: {len(scenario.user_values())}")
print("\n Setup complete! You can now run any of the exploration examples below in any order.")

## Exploration Examples

**The cells below can be run in any order** after completing the setup section above. Each cell demonstrates different ways to explore and analyze scenario data using the pyetm package to connect to the Energy Transition Model's API.

### Basic Scenario Properties

In [ ]:
# Display basic scenario properties
# These properties define the fundamental characteristics of the scenario

print(f"Scenario ID:        {scenario.id}")
print(f"Area Code:          {scenario.area_code}")
print(f"End Year:           {scenario.end_year}")
print(f"Start Year:         {scenario.start_year}")
print(f"Created:            {scenario.created_at}")
print(f"Updated:            {scenario.updated_at}")
print(f"Private:            {scenario.private}")
print(f"Template:           {scenario.template}")
print(f"Source:             {scenario.source}")
print(f"URL:                {scenario.url}")
print(f"Keep Compatible:    {scenario.keep_compatible}")
print(f"Scaling:            {scenario.scaling}")
print(f"Version:            {scenario.version}")

# Show metadata if available
if scenario.metadata:
    print("\nMetadata:")
    print(scenario.metadata)
else:
    print("\nNo additional metadata available")

### Complete Scenario Metadata Export

The `model_dump()` method provides a complete export of all scenario metadata in a structured format. This includes all properties and their current values.

In [ ]:
# Export complete scenario metadata
# model_dump() returns a comprehensive dictionary containing all scenario metadata

full_data = scenario.model_dump()
print(f"Complete scenario metadata ({len(full_data)} fields):")
print("\n" + "-"*70)
print(full_data)

### Exploring User-Modified Values

The `user_values()` method returns a dictionary of all inputs that have been modified from their default values.

In [ ]:
# Display all user-modified input values
# This shows only the inputs that have been changed from their default values
# The format is {input_key: user_value}

user_values = scenario.user_values()
print(f"Found {len(user_values)} user-modified inputs:")
print("\n" + "-"*70)
print(user_values)

### Analyzing Input Properties

Each input in a scenario has various properties that define its behavior:
- `key`: Unique identifier for the input
- `unit`: The unit of measurement (e.g., 'MW', '%', 'PJ')
- `disabled`: Whether the input is currently disabled
- `user`: The value set by the user (if any)
- `default`: The default value for this input

Additional properties for specific input types:
- **Float inputs**: `min` and `max` values defining valid ranges
- **Enumerable inputs**: `permitted_values` showing available options

In [ ]:
# Overview of the input collection
# The inputs property provides access to all scenario inputs through an InputCollection

print(f"Input Collection Overview:")
print(f"  Total inputs: {len(scenario.inputs)}")
print(f"  Input collection type: {type(scenario.inputs)}")
print(f"  User-modified inputs: {len([inp for inp in scenario.inputs if inp.user is not None])}")
print(f"  Disabled inputs: {len([inp for inp in scenario.inputs if inp.disabled])}")

In [ ]:
# Analyze units used across all inputs
# This gives you an overview of what types of measurements are used in the scenario

# Count inputs by unit type
unit_counts = {}
for input in scenario.inputs:
    unit = input.unit or 'No unit'
    unit_counts[unit] = unit_counts.get(unit, 0) + 1

total_inputs = len(scenario.inputs)
print(f"Found {len(unit_counts)} different units used across {total_inputs:,} inputs:")
print("\n" + "-"*70)

# Sort by count (most common first) and show percentages
sorted_units = sorted(unit_counts.items(), key=lambda x: x[1], reverse=True)
for unit, count in sorted_units:
    percentage = (count / total_inputs) * 100
    print(f"{unit:15}: {count:4,} inputs ({percentage:5.1f}%)")

In [ ]:
# Identify disabled inputs
# Disabled inputs are those that do not impact the scenario because of a coupling,
# or if you are not the owner/editor of the scenario

disabled_inputs = [input.key for input in scenario.inputs if input.disabled]
print(f"Found {len(disabled_inputs)} disabled inputs:")
print("\n" + "-"*70)
print(disabled_inputs)

In [ ]:
# Display default values for all inputs
# Default values represent the baseline scenario before user modifications

default_values = { input.key: input.default for input in scenario.inputs }
print(f"Default values for {len(default_values)} inputs:")
print("\n" + "-"*70)
print(default_values)

In [ ]:
# Explore Float input constraints
# Float inputs have minimum and maximum values that define valid ranges
# This information helps to understand input range-limitations and validation rules

from pyetm.models.input import FloatInput

float_inputs = [input for input in scenario.inputs if isinstance(input, FloatInput)]
print(f"Found {len(float_inputs)} float inputs:")
print("\n" + "-"*70)

float_details = [input.model_dump() for input in float_inputs]
print(float_details)

In [ ]:
# Explore Enumerable input options
# Some inputs have a limited set of permitted values (like dropdown selections)
# This is useful for understanding available choices for categorical inputs

from pyetm.models.input import EnumInput

enum_inputs = [input for input in scenario.inputs if isinstance(input, EnumInput)]
print(f"Found {len(enum_inputs)} enumerable inputs:")
print("\n" + "-"*70)

enum_details = [input.model_dump() for input in enum_inputs]
print(enum_details)

### Working with Custom Curves

Custom curves represent time-series data that can be attached to specific inputs. These are typically used for:
- Load profiles (electricity demand over time)
- Production profiles (renewable energy output patterns)
- Availability curves (when technologies are available)
- Price curves (energy prices over time)

The curves contain hourly data points for an entire year (8760 hours).

In [ ]:
# Overview of custom curves collection
# Custom curves are hourly time-series data used to define dynamic behaviour across a year

print(f"Custom Curves Collection Overview:")
print(f"  Collection type: {type(scenario.custom_curves)}")

# Show attached custom curves
attached_curves = list(scenario.custom_curves.attached_keys())
print(f"\nAttached custom curves ({len(attached_curves)}):")
for i, curve_key in enumerate(attached_curves[:10]):
    print(f"  {i+1}. {curve_key}")
if len(attached_curves) > 10:
    print(f"  ... and {len(attached_curves) - 10} more")

In [ ]:
# Example: Display data from a specific curve
# This shows how to access the actual time-series data from a custom curve
# The custom_curve_series() method returns the hourly values for the specified curve
# The curve is stored as a pandas Series

curve_key = 'interconnector_1_import_availability'
if curve_key in scenario.custom_curves.attached_keys():
    curve_data = scenario.custom_curve_series(curve_key)
    print(f"Curve data for '{curve_key}':")
    print(f"  Data points: {len(curve_data)}")
    print(f"  First 10 values: {curve_data.head(10).tolist()}")
    print(f"  Data type: {type(curve_data)}")
    print(f"  Min value: {curve_data.min()}")
    print(f"  Max value: {curve_data.max()}")
    print(f"  Mean value: {curve_data.mean():.4f}")
else:
    print(f"Curve '{curve_key}' not found in this scenario.")
    print("Available curves:", list(scenario.custom_curves.attached_keys()))

In [ ]:
# Analyze all custom curves in the scenario
# This iterates through max 5 attached curves and provides summary statistics

print("Analysis of all custom curves in the scenario:")
print("\n" + "-"*70)

curve_count = 0
for curve_series in scenario.custom_curves_series():
    curve_count += 1
    print(f"\nCurve {curve_count}:")
    print(f"  Length: {len(curve_series)}")
    print(f"  Min: {curve_series.min():.4f}")
    print(f"  Max: {curve_series.max():.4f}")
    print(f"  Mean: {curve_series.mean():.4f}")
    print(f"  Standard deviation: {curve_series.std():.4f}")

    # Stop after first 5 to avoid overwhelming output
    if curve_count >= 5:
        total_curves = len(list(scenario.custom_curves.attached_keys()))
        if total_curves > 5:
            print(f"\n... {total_curves - 5} more curves")
        break

if curve_count == 0:
    print("No custom curves found in this scenario.")

### Working with Carrier Curves

Carrier curves represent time-series data for energy carriers (electricity, heat, gas, etc.) within the energy system. 

Each carrier curve represents an export from your scenario, so they are actually sets of curves, stored as a dataframe where each column is a curve and the index is a time series for each hour.

In [ ]:
# Overview of carrier curves collection
# Carrier curves represent energy flows and storage for different energy carriers

print(f"Carrier Curves Collection Overview:")
print(f"  Collection type: {type(scenario.carrier_curves)}")

# Show first 10 available carrier curve types
attached_carrier_curves = list(scenario.carrier_curves.attached_keys())
print(f"\nAttached carrier curves ({len(attached_carrier_curves)}):")
for i, curve_key in enumerate(attached_carrier_curves[:10]):
    print(f"  {i+1}. {curve_key}")
if len(attached_carrier_curves) > 10:
    print(f"  ... {len(attached_carrier_curves) - 10} more")

In [ ]:
# Analyze all carrier curves in the scenario
# This provides a comprehensive overview of carrier curve behaviour and format

print("Analysis of all carrier curves in the scenario:")
print("\n" + "-"*70)

curve_count = 0
attached_curve_keys = list(scenario.carrier_curves.attached_keys())
for curve_data in scenario.carrier_curves_series():
    curve_name = attached_curve_keys[curve_count] if curve_count < len(attached_curve_keys) else f"Unknown Curve {curve_count + 1}"
    curve_count += 1
    print(f"\n{curve_name}:")

    if curve_data is not None and not curve_data.empty:
        print(f"  Shape (rows × columns): {curve_data.shape}")
        print(f"  Columns: {list(curve_data.columns)}")

        # For DataFrames, show summary stats differently
        numeric_cols = curve_data.select_dtypes(include=[float, int]).columns
        if len(numeric_cols) > 0:
            print(f"  Summary for numeric columns:")
            for col in numeric_cols[:3]:  # Show first 3 columns to avoid clutter
                col_data = curve_data[col]
                print(f"    {col}:")
                print(f"      Min: {float(col_data.min()):.4f}")
                print(f"      Max: {float(col_data.max()):.4f}")
                print(f"      Mean: {float(col_data.mean()):.4f}")
                print(f"      Std: {float(col_data.std()):.4f}")
            if len(numeric_cols) > 3:
                print(f"    ... and {len(numeric_cols) - 3} more columns")
        else:
            print(f"  No numeric data available")
    else:
        print(f"  No data available")

    # Stop after first 5 to avoid overwhelming output
    if curve_count >= 5:
        total_curves = len(attached_curve_keys)
        if total_curves > 5:
            print(f"\n... and {total_curves - 5} more carrier curve exports")
        break

if curve_count == 0:
    print("No carrier curves found in this scenario.")

### Exploring Sortables

Sortables represent ordered lists of technologies or components within the energy system. They define priority orders for:
- Merit order
- Forecast order
- Heat network order
- Hydrogen supply/demand

The order of items in sortables affects how the energy system model calculates results.

In [ ]:
# Overview of sortables collection
# Sortables define ordering and priority for various energy system components

print(f"Sortables Overview:")
print(f"  Data type: {type(scenario.sortables)}")

# Show all sortables
sortable_keys = list(scenario.sortables.as_dict().keys())
print(f"\nSortables ({len(sortable_keys)}):")
for i, sortable_key in enumerate(sortable_keys):
    print(f"  {i+1}. {sortable_key}")

In [ ]:
# Display sortable configurations
# Sortables define the order/priority of different technologies in the energy system
# For example, merit order determines which power plants are dispatched first
# The order directly affects energy system calculations and results

sortables_data = scenario.sortables.as_dict()
print(f"Found {len(sortables_data)} sortable orders:")
print("\n" + "-"*70)
print(sortables_data)

In [ ]:
# Analysis of each sortable category
# This examines the structure and content of each sortable configuration

sortables_data = scenario.sortables.as_dict()
print("Detailed analysis of sortables:")
print("\n" + "-"*70)

for category, sortables in sortables_data.items():
    print(f"\n{category.upper()}:")

    if sortables:
        print(f"  Number of items: {len(sortables)}")
        print(f"  Sortables:")
        for i, order in enumerate(sortables):
            print(f"    {i+1}. {order}")

### Working with Gqueries

Gqueries allow you to extract specific calculated values from the ETM.

You can request multiple queries and execute them together for efficiency.

In [ ]:
scenario.add_queries([
    "dashboard_emissions",
    "dashboard_total_costs",
    "dashboard_renewability"
])

print(f"Added {len(scenario._queries.query_keys())} queries:")
for i, query in enumerate(scenario._queries.query_keys(), 1):
    print(f"  {i}. {query}")

print(f"\nQueries ready: {scenario._queries.is_ready()}")

print("Gqueries Overview:")
print(f"  Queries requested: {scenario.queries_requested()}")

if scenario.queries_requested():
    print(f"  Query keys: {scenario._queries.query_keys()}")
    print(f"  Queries ready: {scenario._queries.is_ready()}")

    # Get results if available
    results = scenario.results()
    if results is not None and not results.empty:
        print(f"\nQuery Results:")
        print(f"  Results shape: {results.shape}")
        print(f"  Columns: {list(results.columns)}")
        print(f"   ")
        print(results.head())
    else:
        print("\nNo query results available")

### Handling Warnings and Errors

The scenario object can accumulate warnings during data fetching and processing. These warnings provide important information about non-breaking issues with data quality, your API configuration, or service-level issues.

In [ ]:
# Check for warnings and errors in the scenario
# Warnings can accumulate from API calls, data validation, or processing issues

# Check if the scenario object has any warnings
if hasattr(scenario, 'warnings') and scenario.warnings:
    print(f"  Total warnings: {len(scenario.warnings)}")
    print("\nWarnings:")
    for i, warning in enumerate(scenario.warnings, 1):
        print(f"  {i}. {warning}")
else:
    print(f"  No warnings found")